In [1]:
# ==========================================
# Median Elimination Algorithm and Utilities
# ==========================================

import numpy as np
from math import log, ceil
from collections import defaultdict

def median_elimination(arm_pulls, epsilon=0.1, delta=0.05):
    """
    Median Elimination (PAC-style) for best-arm identification.

    Args:
        arm_pulls (dict): {arm_index: [(reward, cost), ...]}
        epsilon (float): Accuracy parameter (ε-optimal arm).
        delta (float): Confidence parameter (1 - δ success).

    Returns:
        chosen_arm (int): Selected ε-optimal arm.
        total_pulls (int): Total number of pulls used.
        total_cost (float): Total cost incurred.
        confidence (float): Lower bound on probability of correctness.
        stopping_early (bool): True if terminated due to data exhaustion.
    """
    S = list(arm_pulls.keys())
    ε_l = epsilon / 4
    δ_l = delta / 2
    total_pulls = 0
    total_cost = 0
    sample_means = {}
    num_pulls = defaultdict(int)
    stopping_early = False

    while len(S) > 1:
        t_l = ceil((4 / (ε_l ** 2)) * log(3 / δ_l))

        max_available = min([len(arm_pulls[arm]) - num_pulls[arm] for arm in S])
        if max_available <= 0:
            stopping_early = True
            break
        t_l = min(t_l, max_available)

        empirical_means = {}
        for arm in S:
            start = num_pulls[arm]
            pulls = arm_pulls[arm][start:start + t_l]
            rewards = [r for r, _ in pulls]
            costs = [c for _, c in pulls]

            if not rewards:
                continue

            avg_reward = np.mean(rewards)
            empirical_means[arm] = avg_reward
            sample_means[arm] = avg_reward
            num_pulls[arm] += len(rewards)
            total_pulls += len(rewards)
            total_cost += sum(costs)

        if not empirical_means:
            stopping_early = True
            break

        median_value = np.median(list(empirical_means.values()))
        S = [arm for arm in S if empirical_means.get(arm, 0) >= median_value]

        ε_l *= 0.75
        δ_l *= 0.5

    chosen_arm = S[0] if S else None
    confidence = hoeffding_confidence(chosen_arm, sample_means, num_pulls)
    return chosen_arm, total_pulls, total_cost, confidence, stopping_early


def hoeffding_confidence(chosen_arm, sample_means, num_pulls):
    """
    Returns a conservative lower bound on the confidence that chosen_arm is ε-optimal.
    """
    chosen_mean = sample_means.get(chosen_arm, 0)
    confidences = []
    for arm, mean in sample_means.items():
        if arm == chosen_arm:
            continue
        delta = chosen_mean - mean
        if delta <= 0:
            confidences.append(0.0)
        else:
            n = min(num_pulls.get(arm, 1), num_pulls.get(chosen_arm, 1))
            conf = 1 - np.exp(-0.5 * n * delta ** 2)
            confidences.append(conf)
    return min(confidences) if confidences else 1.0


In [2]:
# ================================================
# Synthetic Data Generator (Mixture Normal Gaps)
# ================================================

import random
import numpy as np
from collections import defaultdict
from tqdm.auto import tqdm

# Experiment-level constants
NUM_CASES           = 5_000         # number of bandit instances to simulate
NUM_ARMS            = 6             # arms per bandit
NUM_PULLS_PER_ARM   = 1_500         # horizon per arm
COST_RANGE          = (0.05, 0.20)  # per-pull cost range
DELTA               = 0.05          # ME confidence parameter

# Mixture specification: (reward-gap, probability)
#  ➜ 99 % of cases have ≈0.10 gap; 1 % have ≈0.20 gap
MIXTURE_GAPS = [(0.10, 0.99),
                (0.20, 0.01)]

# ---------- helper to sample a gap according to the mixture ----------
def sample_gap() -> float:
    r = random.random()
    cumulative = 0.0
    for gap, prob in MIXTURE_GAPS:
        cumulative += prob
        if r <= cumulative:
            return gap
    # Numerical fallback (rare if probs sum to 1)
    return MIXTURE_GAPS[-1][0]

# ---------- single bandit instance ----------
def generate_case():
    """
    Generate one 6-arm bandit instance whose arm rewards are Normal-distributed
    BUT whose *reward gap* (best-arm mean minus each sub-optimal arm mean)
    is drawn from the specified mixture.
    """
    gap = sample_gap()                              # 0.10 w.p. 0.99, 0.20 w.p. 0.01
    best_mean = round(random.uniform(0.60, 0.90), 3)  # overall best arm’s mean

    # For every sub-optimal arm, sample a mean at least `gap` below the best,
    # with a little extra randomness so that gaps aren’t all identical.
    other_means = [
        best_mean - random.uniform(gap, gap + 0.10)
        for _ in range(NUM_ARMS - 1)
    ]

    # Shuffle so the best arm index is not always 0
    all_means = [best_mean] + other_means
    random.shuffle(all_means)

    # Per-arm reward standard deviations
    std_devs = [
        round(random.uniform(0.05, 0.15), 3)
        for _ in range(NUM_ARMS)
    ]

    # Materialise pulls for each arm
    pulls = defaultdict(list)
    for arm in range(NUM_ARMS):
        rewards = np.random.normal(
            loc   = all_means[arm],
            scale = std_devs[arm],
            size  = NUM_PULLS_PER_ARM
        )
        costs = np.random.uniform(*COST_RANGE, size=NUM_PULLS_PER_ARM)
        pulls[arm] = list(zip(rewards, costs))

    return pulls, all_means, std_devs, gap

# ---------- experiment runner ----------
def run_median_elimination_experiments(num_cases: int = NUM_CASES):
    """
    Run Median Elimination on `num_cases` mixture-gap bandits.
    Returns:
        me_results  – dict of performance metrics
        all_means   – list of all arm means across cases (for stats)
        all_stds    – list of all arm std-devs across cases
        gaps_seen   – list of gap values actually sampled
    """
    me_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    all_means, all_stds, gaps_seen = [], [], []

    for _ in tqdm(range(num_cases)):
        arm_pulls, means, stds, gap = generate_case()
        gaps_seen.append(gap)
        all_means.extend(means)
        all_stds.extend(stds)
        best_arm = int(np.argmax(means))

        me_arm, mp, mc, mcf, early_m = median_elimination(
            arm_pulls,
            epsilon = 0.10,
            delta   = DELTA
        )

        me_results['pulls'].append(mp)
        me_results['cost'].append(mc)
        me_results['confidence'].append(mcf)
        me_results['correct'] += (me_arm == best_arm)
        me_results['early']   += early_m

    return me_results, all_means, all_stds, gaps_seen

# ---------- pretty printer ----------
def print_me_summary(results_tuple, num_cases):
    results, means, stds, gaps = results_tuple

    def summarize(res):
        cost_std_err = np.std(res['cost']) / np.sqrt(num_cases)
        confidence_std_err = np.std(res['confidence']) / np.sqrt(num_cases)
        return {
            'Avg Pulls'            : np.mean(res['pulls']),
            'Avg Cost'             : np.mean(res['cost']),
            'Standard Error in Cost (%)': 100 * cost_std_err / np.mean(res['cost']) if np.mean(res['cost']) > 0 else 0,
            'Avg Confidence'       : np.mean(res['confidence']),
            'Standard Error in Confidence (%)': 100 * confidence_std_err / np.mean(res['confidence']) if np.mean(res['confidence']) > 0 else 0,
            'Accuracy (%)'         : 100 * res['correct'] / num_cases,
            'Stopping Early (%)'   : 100 * res['early']   / num_cases
        }

    me_stats = summarize(results)
    print("\n=== Median Elimination (Mixture Normal) ===")
    for k, v in me_stats.items():
        print(f"{k}: {v:.2f}")

    print("\n=== Reward Distribution Summary ===")
    print(f"Mean of reward means   : {np.mean(means):.4f}")
    print(f"Mean of std deviations : {np.mean(stds):.4f}")

    print("\n=== Gap Mixture Check ===")
    unique, counts = np.unique(gaps, return_counts=True)
    for g, c in zip(unique, counts):
        print(f"Gap {g:0.2f}: {100*c/num_cases:.2f}% of cases")

# ------------
#  Run experiment
# ------------
if __name__ == "__main__":
    results = run_median_elimination_experiments()
    print_me_summary(results, NUM_CASES)

  0%|          | 0/5000 [00:00<?, ?it/s]


=== Median Elimination (Mixture Normal) ===
Avg Pulls: 9000.00
Avg Cost: 1125.00
Standard Error in Cost (%): 0.01
Avg Confidence: 0.33
Standard Error in Confidence (%): 2.02
Accuracy (%): 32.88
Stopping Early (%): 100.00

=== Reward Distribution Summary ===
Mean of reward means   : 0.6250
Mean of std deviations : 0.1000

=== Gap Mixture Check ===
Gap 0.10: 98.80% of cases
Gap 0.20: 1.20% of cases


In [3]:
# ================================================
# Synthetic Data Generator (Mixture Normal Gaps)
# ================================================

import random
import numpy as np
from collections import defaultdict
from tqdm.auto import tqdm

# Experiment-level constants
NUM_CASES           = 5_000         # number of bandit instances to simulate
NUM_ARMS            = 6             # arms per bandit
NUM_PULLS_PER_ARM   = 1_500         # horizon per arm
COST_RANGE          = (0.05, 0.20)  # per-pull cost range
DELTA               = 0.05          # ME confidence parameter

# Mixture specification: (reward-gap, probability)
#  ➜ 99 % of cases have ≈0.10 gap; 1 % have ≈0.20 gap
MIXTURE_GAPS = [(0.10, 0.90),
                (0.20, 0.10)]

# ---------- helper to sample a gap according to the mixture ----------
def sample_gap() -> float:
    r = random.random()
    cumulative = 0.0
    for gap, prob in MIXTURE_GAPS:
        cumulative += prob
        if r <= cumulative:
            return gap
    # Numerical fallback (rare if probs sum to 1)
    return MIXTURE_GAPS[-1][0]

# ---------- single bandit instance ----------
def generate_case():
    """
    Generate one 6-arm bandit instance whose arm rewards are Normal-distributed
    BUT whose *reward gap* (best-arm mean minus each sub-optimal arm mean)
    is drawn from the specified mixture.
    """
    gap = sample_gap()                              # 0.10 w.p. 0.99, 0.20 w.p. 0.01
    best_mean = round(random.uniform(0.60, 0.90), 3)  # overall best arm’s mean

    # For every sub-optimal arm, sample a mean at least `gap` below the best,
    # with a little extra randomness so that gaps aren’t all identical.
    other_means = [
        best_mean - random.uniform(gap, gap + 0.10)
        for _ in range(NUM_ARMS - 1)
    ]

    # Shuffle so the best arm index is not always 0
    all_means = [best_mean] + other_means
    random.shuffle(all_means)

    # Per-arm reward standard deviations
    std_devs = [
        round(random.uniform(0.05, 0.15), 3)
        for _ in range(NUM_ARMS)
    ]

    # Materialise pulls for each arm
    pulls = defaultdict(list)
    for arm in range(NUM_ARMS):
        rewards = np.random.normal(
            loc   = all_means[arm],
            scale = std_devs[arm],
            size  = NUM_PULLS_PER_ARM
        )
        costs = np.random.uniform(*COST_RANGE, size=NUM_PULLS_PER_ARM)
        pulls[arm] = list(zip(rewards, costs))

    return pulls, all_means, std_devs, gap

# ---------- experiment runner ----------
def run_median_elimination_experiments(num_cases: int = NUM_CASES):
    """
    Run Median Elimination on `num_cases` mixture-gap bandits.
    Returns:
        me_results  – dict of performance metrics
        all_means   – list of all arm means across cases (for stats)
        all_stds    – list of all arm std-devs across cases
        gaps_seen   – list of gap values actually sampled
    """
    me_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    all_means, all_stds, gaps_seen = [], [], []

    for _ in tqdm(range(num_cases)):
        arm_pulls, means, stds, gap = generate_case()
        gaps_seen.append(gap)
        all_means.extend(means)
        all_stds.extend(stds)
        best_arm = int(np.argmax(means))

        me_arm, mp, mc, mcf, early_m = median_elimination(
            arm_pulls,
            epsilon = 0.10,
            delta   = DELTA
        )

        me_results['pulls'].append(mp)
        me_results['cost'].append(mc)
        me_results['confidence'].append(mcf)
        me_results['correct'] += (me_arm == best_arm)
        me_results['early']   += early_m

    return me_results, all_means, all_stds, gaps_seen

# ---------- pretty printer ----------
def print_me_summary(results_tuple, num_cases):
    results, means, stds, gaps = results_tuple

    def summarize(res):
        cost_std_err = np.std(res['cost']) / np.sqrt(num_cases)
        confidence_std_err = np.std(res['confidence']) / np.sqrt(num_cases)
        return {
            'Avg Pulls'            : np.mean(res['pulls']),
            'Avg Cost'             : np.mean(res['cost']),
            'Standard Error in Cost (%)': 100 * cost_std_err / np.mean(res['cost']) if np.mean(res['cost']) > 0 else 0,
            'Avg Confidence'       : np.mean(res['confidence']),
            'Standard Error in Confidence (%)': 100 * confidence_std_err / np.mean(res['confidence']) if np.mean(res['confidence']) > 0 else 0,
            'Accuracy (%)'         : 100 * res['correct'] / num_cases,
            'Stopping Early (%)'   : 100 * res['early']   / num_cases
        }

    me_stats = summarize(results)
    print("\n=== Median Elimination (Mixture Normal) ===")
    for k, v in me_stats.items():
        print(f"{k}: {v:.2f}")

    print("\n=== Reward Distribution Summary ===")
    print(f"Mean of reward means   : {np.mean(means):.4f}")
    print(f"Mean of std deviations : {np.mean(stds):.4f}")

    print("\n=== Gap Mixture Check ===")
    unique, counts = np.unique(gaps, return_counts=True)
    for g, c in zip(unique, counts):
        print(f"Gap {g:0.2f}: {100*c/num_cases:.2f}% of cases")

# ------------
#  Run experiment
# ------------
if __name__ == "__main__":
    results = run_median_elimination_experiments()
    print_me_summary(results, NUM_CASES)

  0%|          | 0/5000 [00:00<?, ?it/s]


=== Median Elimination (Mixture Normal) ===
Avg Pulls: 9000.00
Avg Cost: 1124.96
Standard Error in Cost (%): 0.01
Avg Confidence: 0.33
Standard Error in Confidence (%): 2.01
Accuracy (%): 33.18
Stopping Early (%): 100.00

=== Reward Distribution Summary ===
Mean of reward means   : 0.6160
Mean of std deviations : 0.1002

=== Gap Mixture Check ===
Gap 0.10: 89.64% of cases
Gap 0.20: 10.36% of cases
